# Integración de los datos en un único fichero CSV para ser empleados por la RNA
Se realiza la integración en un único fichero CSV, de forma que se debe indicar de qué sensor se tratan los datos, además de integrar los datos del pluviómetro.

In [15]:
import pandas as pd

import os 

from global_variables import *
from env_paths import *

In [16]:
sensor_files = [file for file in os.listdir(SENSOR_FILES_DIRECTORY) if ".csv" in file]
pluviometer_files = [file for file in os.listdir(PLUVIOMETER_FILES_DIRECTORY) if ".csv" in file]

In [17]:
pluviometer = pd.read_csv(CLASSIFIED_PLUVIOMETER_PATH + pluviometer_files[0])
pluviometer['ts'] = pd.to_datetime(pluviometer['ts'], unit="ns")

In [18]:
final_df = pd.DataFrame()
for n_sensor, file in enumerate(sensor_files):
    df = pd.read_csv(CLASSIFIED_SENSOR_PATH + file)

    df['ts'] = pd.to_datetime(df['ts'], unit='ns')
    df = pd.merge(df, pluviometer, on='ts', how='left')
    df['sensor'] = pd.Series([n_sensor] * df.shape[0])

    # Introducimos los datos del pluviometro previamente tratados para realizar un estudio
    # de la correlacion con el resto de variables (en la mayoria de los casos esta variable
    # no supera el 20% de correlacion con el resto en niungun sensor)
    df['pluv_deltaMM'] = df['pluv_deltaMM'].interpolate()
    
    print(f"Matriz de correlación de {file}")
    print(df.corr(numeric_only=True))
    print()

    df = df.drop('pluv_deltaMM', axis=1)

    # Concatenamos el conjunto de datos del sensor actual con el resto para tener los datos
    # integrados
    final_df = pd.concat([final_df, df])
final_df.to_csv(INTEGRATED_FILES_PATH + "/data.csv", index = False)

Matriz de correlación de sensor1.csv
                 t_ext     h_ext   t_C_cal   h_C_cal   t_L_cal   h_L_cal  \
t_ext         1.000000 -0.835807  0.867878 -0.132617  0.843794 -0.302618   
h_ext        -0.835807  1.000000 -0.644912  0.093685 -0.624613  0.165570   
t_C_cal       0.867878 -0.644912  1.000000 -0.118532  0.990982 -0.432535   
h_C_cal      -0.132617  0.093685 -0.118532  1.000000 -0.094590 -0.003731   
t_L_cal       0.843794 -0.624613  0.990982 -0.094590  1.000000 -0.477522   
h_L_cal      -0.302618  0.165570 -0.432535 -0.003731 -0.477522  1.000000   
pluv_deltaMM -0.131528  0.204985 -0.126116 -0.084094 -0.127983  0.099010   
sensor             NaN       NaN       NaN       NaN       NaN       NaN   

              pluv_deltaMM  sensor  
t_ext            -0.131528     NaN  
h_ext             0.204985     NaN  
t_C_cal          -0.126116     NaN  
h_C_cal          -0.084094     NaN  
t_L_cal          -0.127983     NaN  
h_L_cal           0.099010     NaN  
pluv_deltaMM      1